In [ ]:
import sys
import os

from IPython.display import display, Markdown, Code

model = "gemini-2.5-pro"

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name=model)

with open("../documents/db_schema.sql", "r", encoding="utf-8") as f:
    sql_schema = f.read()

if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

In [ ]:
# try:
#     with open("../app/scripts/main_in_memory.py", "r", encoding="utf-8") as f:
#         in_memory_api_code = f.read()
# except FileNotFoundError:
#     in_memory_api_code = None

in_memory_api_prompt = f"""You are a senior Python developer with expertise in FastAPI and in-memory databases.
- Using the provided SQL schema, generate a complete FastAPI application that implements full CRUD operations for the endpoints in the schema using Pydantic models and non-deprecated methods.
- Use an in-memory data structure (like a list or dictionary) to store user data.
- Do not include any response other than valid Python code with comprehensive docstrings.
- Ensure the code is ready to run as a standalone FastAPI application.
- Use Pydantic models for request validation and response serialization.
- Ensure constraints are propertly implemented, where applicable.
- Include error handling for cases like user not found, invalid input data, and duplicate entries.
- Include a name==main block to run the app with Uvicorn on port 8081.
- If available, use the previously generated code as a reference.
    - Make any changes noted in comments at the end of the generated code file.

<SQL Schema>
{sql_schema}
</SQL Schema>

<Previously Generated Code>
{in_memory_api_code if 'in_memory_api_code' in locals() else "Code not available."}
</Previously Generated Code>
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')

    with open("../app/scripts/main_in_memory.py", "w", encoding="utf-8") as f:
        f.write(cleaned_code)
    
else:
    print("Skipping API generation because schema is missing.")

In [ ]:
db_code_prompt = f"""
You are a senior Python developer with expertise in SQLAlchemy.
- Using the provided SQLite schema, generate a complete FastAPI application that implements full CRUD operations for the tables using Pydantic models and non-deprecated methods.
- Using the provided SQLite schema, generate ONLY two separate Python files:
    1. 'models.py': Contains SQLAlchemy classes for all tables in the database. Each class should be prepended by 'sqa_'.
    2. 'database.py': Contains boilerplate code to create a database engine, session maker, and a dependency function ("get_db") to get a database session.
    3. Do not include any other files or code.
- Do not include any text other than valid Python code with comprehensive docstrings according to these instructions.
- Ensure that each code block is ready to run in Python.

<SQLite Schema>
{sql_schema}
</SQLite Schema>
"""

print("Generating SQLAlchemy Models and Session Code...")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    display(Code(generated_db_code, language="python"))
else:
    print("Skipping DB code generation because schema is missing.")

In [ ]:
with open("../app/scripts/main_in_memory.py", "r", encoding="utf-8") as f:
    generated_api_code = f.read()

with open("../app/scripts/models.py", "r", encoding="utf-8") as f:
    generated_models_code = f.read()

with open("../app/scripts/database.py", "r", encoding="utf-8") as f:
    generated_database_code = f.read()

stitch_prompt = f"""
You are a senior Python developer with expertise in FastAPI and SQLAlchemy.
- Your task is to stitch two provided python files into a single FastAPI application to make a complete CRUD API.
- Replace any in-memory data structures with SQLAlchemy ORM queries to interact with a SQLite database.
    - The first file is the main application file with all the CRUD operations and Pydantic models.
    - The second file contains the SQLAlchemy models.
    - The third file contains the database session setup.
- Please combine them into a single cohesive FastAPI application.
- Refactor the code as needed to ensure it is clean, efficient, and follows best practices to properly perform CRUD operations on the database using SQLAlchemy.
- Ensure the code is ready to run as a standalone FastAPI application.
- Ensure there is a name==main block to run the app with Uvicorn on port 8081.
- Do not include any text other than valid Python code with comprehensive docstrings according to these instructions.

<Main Application Code>
{generated_api_code}
</Main Application Code>

<Models Code>
{generated_models_code}
</Models Code>

<Database Session Code>
{generated_database_code}
</Database Session Code>
"""

print("Stitching API and Models Code...")
if generated_api_code and generated_models_code:
    stitched_code = f"{generated_api_code}\n{generated_models_code}"

    generated_stitched_code = get_completion(stitch_prompt, client, model_name, api_provider)
    cleaned_stitched_code = clean_llm_output(generated_stitched_code, language='python')
    with open("../app/scripts/main.py", "w", encoding="utf-8") as f:
        f.write(cleaned_stitched_code)